In [6]:
import simpy
import numpy as np

# -----------------------------
# Configuration
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) # simulation runs for 3 years, measured in weeks
SEED = 42

# -----------------------------
# Input
TOTAL_PATIENTS_YEAR = 95201   # people “show up” per year
YES_PATIENTS_YEAR = 84557     # only YES_PATIENTS_YEAR actually start therapy
PATIENT_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR # random arrival starts therapy

# Therapy duration (in weeks)
SESSIONS_REQUIRED = 24        # Therapy needs 24 sessions
WORKING_WEEKS = 42            # 42 working weeks/year (vacation/sick leave removed)
TIME_STRETCH = WEEKS_PER_YEAR / WORKING_WEEKS # “session week” is stretched by 52/42
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH # a therapy course occupies a slot for about 29.7 week

# Capacity calculation
# theoretical maximum number of parallel therapy slots based on staff groups
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 15
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME # ~102.060

# Slots needed for 100% coverage
# Each year YES_PATIENTS_YEAR therapy starts
# Patient start occupies a slot for REAL_DURATION_WEEKS
# Divide by 52 to convert to “average concurrent slots needed”
REQUIRED_SLOTS_SATURATION = (YES_PATIENTS_YEAR * REAL_DURATION_WEEKS) / WEEKS_PER_YEAR

# Break-even factor
# theoretical capacity you’d need so that capacity equals demand (100% coverage)
# converts that to an absolute number of slots 
BREAK_EVEN_FACTOR = REQUIRED_SLOTS_SATURATION / THEORETICAL_CAPACITY

# Scenario adjustment
# 1.00 = Perfect (no waiting lines)
# 0.95 = 5% too few slots (waiting lines build up)
# 0.90 = 10% too few slots (severe congestion)
SCENARIO_ADJUSTMENT = 0.90 

# Effective simulation capacity
# number of slots to use
REAL_EFFICIENCY_FACTOR = BREAK_EVEN_FACTOR * SCENARIO_ADJUSTMENT
# REAL_CAPACITY = number of parallel therapy places
REAL_CAPACITY = int(THEORETICAL_CAPACITY * REAL_EFFICIENCY_FACTOR)

# Demand growth (arrival rate)
# At time 0, expected arrivals per week are TOTAL/52
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005  # Demand then grows continuously

stats_wait_times = []    # waiting times (in weeks) for patients who successfully get therapy


# -----------------------------
# Simulation processes
# -----------------------------

def patient_process(env, name, therapy_slots, dropped_counter):
    """
    Simulates the lifecycle of a single patient seeking therapy.
    
    This process models the flow from arrival to potentially finishing therapy.
    1. It checks if the patient actually starts therapy (based on `PATIENT_START_THERAPY` probability).
    2. If yes, the patient requests a `therapy_slot` from the resource.
    3. The patient waits (yields) until a slot becomes available.
    4. The waiting time is recorded.
    5. The patient occupies the slot for `REAL_DURATION_WEEKS`.

    Variables used:
    - env: The SimPy environment object.
    - name: String identifier for the patient.
    - therapy_slots: The SimPy Resource representing available therapy spots.
    - dropped_counter: Dictionary to track patients who do not start therapy.
    - rng (Global): Random number generator for probability checks.
    - PATIENT_START_THERAPY (Global): Probability threshold for starting therapy.
    - stats_wait_times (Global): List to store waiting time results.
    - REAL_DURATION_WEEKS (Global): Duration the slot is occupied.
    """
    arrival_time = env.now  # patient arrives
    
    # With probability (1 - P_START_THERAPY) they do not start therapy and are counted as dropped
    if rng.random() > PATIENT_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    # request a therapy slot
    # If none are free, they wait until one is available
    with therapy_slots.request() as req:
        yield req 
        
        # Platz bekommen -> Wartezeit speichern
        stats_wait_times.append(env.now - arrival_time) # waiting time is env.now - arrival_time.
        
        # Patient occupies the slot for REAL_DURATION_WEEKS
        yield env.timeout(REAL_DURATION_WEEKS)


def pre_fill_slot(env, therapy_slots, rng):
    """
    Occupies a therapy slot at the beginning of the simulation (Warm-up phase).
    
    To avoid starting with an empty clinic (which would result in artificially low waiting times 
    for the first patients), this function blocks slots for a random duration between 0 
    and the full therapy length. This simulates ongoing therapies that started before t=0.

    Variables used:
    - env: The SimPy environment object.
    - therapy_slots: The SimPy Resource to be pre-filled.
    - rng: Random number generator for the uniform distribution.
    - REAL_DURATION_WEEKS (Global): Maximum duration for the warm-up period.
    """
    with therapy_slots.request() as req:
        yield req
        # Each pre-filled slot is released after a random “remaining duration” 
        # between 0 and the full therapy length
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))


def setup(env, capacity, rng):
    """
    Sets up the simulation environment and generates the patient flow.
    
    1. Creates the `therapy_slots` Resource with the calculated capacity.
    2. Runs the warm-up phase by calling `pre_fill_slot` for all available slots.
    3. Continuously generates new patients following a Poisson process.
    4. Increases the arrival rate (`current_rate`) over time to simulate demand growth.

    Variables used:
    - env: The SimPy environment object.
    - capacity: Total number of available therapy slots.
    - rng: Random number generator.
    - INITIAL_WEEKLY_RATE (Global): Starting value for patient arrivals per week.
    - SIMULATION_DURATION (Global): The total time the simulation runs.
    - GROWTH_PER_WEEK (Global): Factor for exponential demand growth.
    """
    # number of parallel slots
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # Arrivals 
    # Interarrival times are exponential 
    # arrivals follow a Poisson process with rate current_rate
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        # Exponentielles Wachstum der Nachfrage
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))


# -----------------------------
# MAIN
# -----------------------------

if __name__ == "__main__":
    # Prints summary
    print(f"----- CAPACITY CALCULATION -----")
    print(f"Demand (Yes Patients):       {YES_PATIENTS_YEAR}")
    print(f"Actual therapy duration:     {REAL_DURATION_WEEKS:.2f} weeks")
    print(f"Required slots (100%):       {int(REQUIRED_SLOTS_SATURATION)}")
    print(f"Theoretical slots:           {THEORETICAL_CAPACITY}")
    print(f"-"*60)
    print(f"Break-even factor:           {BREAK_EVEN_FACTOR:.4f} (this equals 100%)")
    print(f"-"*60)
    print(f"Scenario setting:            {SCENARIO_ADJUSTMENT*100:.1f}% of demand (under-supply)")
    print(f"-> Applied efficiency factor:{REAL_EFFICIENCY_FACTOR:.4f}")
    print(f"-> Effective capacity:       {REAL_CAPACITY} slots")
    print(f"="*60)
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n----- RESULTS -----")
    if n_started > 0:
        mean_weeks = np.mean(waits)
        median_weeks = np.median(waits)
        p90_weeks = np.quantile(waits, 0.90)

        print(f"Patients starting therapy:   {n_started}")
        print(f"Average waiting time:        {mean_weeks*7:.1f} days")
        print(f"Median waiting time:         {median_weeks*7:.1f} days")
        print(f"P90 waiting time (risk):     {p90_weeks*7:.1f} days")
        print(f"Maximum waiting time:        {np.max(waits)*7:.1f} days")
    else:
        print("No data available.")

----- CAPACITY CALCULATION -----
Demand (Yes Patients):       84557
Actual therapy duration:     29.71 weeks
Required slots (100%):       48318
Theoretical slots:           102060
------------------------------------------------------------
Break-even factor:           0.4734 (this equals 100%)
------------------------------------------------------------
Scenario setting:            90.0% of demand (under-supply)
-> Applied efficiency factor:0.4261
-> Effective capacity:       43486 slots

----- RESULTS -----
Patients starting therapy:   228337
Average waiting time:        142.6 days
Median waiting time:         127.4 days
P90 waiting time (risk):     300.3 days
Maximum waiting time:        350.0 days
